In [45]:
import os
import pyodbc

import numpy as np 
import matplotlib as plt 
import pandas as pd
from sklearn import preprocessing

In [46]:
# make rule grape with deviation, mean from actucater data
# like SPC

In [ ]:
# OP_NO,STATS_BIT를 받아와야함

In [204]:
LINE_group = OrderedDict()
OP_group = OrderedDict()
STATUS_BIT_group_OP10 = OrderedDict()
STATUS_BIT_group_OP20 = OrderedDict()
STATUS_BIT_group_OP30 = OrderedDict()
STATUS_BIT_group_OP40 = OrderedDict()
STATUS_BIT_group_OP50 = OrderedDict()

In [205]:
def CalActuater(OP_NO,STATS_BIT):
    try:
        conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                          "Server=MDPLP1546701\\SQLEXPRESS;"
                          "Database=CETA_LMS;"
                          "Trusted_Connection=yes;")
        cursor = conn.cursor()  
        cursor.execute(''' SELECT TOP (1000) [MCH_STATUS_SUMMARY].[OP_NO],
            MCH_STATUS_SUMMARY.STATUS_BIT,DATEDIFF([MS],[STATUS_SDATE],
            [STATUS_EDATE]) as mstime ,STD_MCH_STATUS_INFO.STATUS_DESC 
            FROM [CETA_LMS].[dbo].[MCH_STATUS_SUMMARY] left outer join CETA_LMS.dbo.STD_MCH_STATUS_INFO 
            on MCH_STATUS_SUMMARY.STATUS_BIT = STD_MCH_STATUS_INFO.STATUS_IO_NO and MCH_STATUS_SUMMARY.OP_NO =STD_MCH_STATUS_INFO.OP_NO
            where MCH_STATUS_SUMMARY.STATUS_BIT= ('%s') and MCH_STATUS_SUMMARY.OP_NO=('%s'); '''  %(STATUS_BIT,OP_NO) )  

        rs = cursor.fetchall()
        mslist=list()

        descript = rs[1][3]
        for i in rs:
            #print(i[2])
            mslist.append(i[2]) # 2번자리는 변동없음, but sql문에 op_no와 bit가 입력변수로 들어가야함 %s형식, 익산것 참조
            # status_bit 별로 추가 


        aa=pd.DataFrame(mslist)
        mean=int(aa.mean())
        std=aa.std()

        # 아래부터 NG여부 판단
        cursor = conn.cursor()  
        cursor.execute(''' SELECT TOP (1) [MCH_STATUS_SUMMARY].[OP_NO],
            MCH_STATUS_SUMMARY.STATUS_BIT,DATEDIFF([MS],[STATUS_SDATE],
            [STATUS_EDATE]) as mstime ,STD_MCH_STATUS_INFO.STATUS_DESC 
            FROM [CETA_LMS].[dbo].[MCH_STATUS_SUMMARY] left outer join CETA_LMS.dbo.STD_MCH_STATUS_INFO 
            on MCH_STATUS_SUMMARY.STATUS_BIT = STD_MCH_STATUS_INFO.STATUS_IO_NO and MCH_STATUS_SUMMARY.OP_NO =STD_MCH_STATUS_INFO.OP_NO
            where MCH_STATUS_SUMMARY.STATUS_BIT= ('%s') and MCH_STATUS_SUMMARY.OP_NO=('%s'); '''  %(STATUS_BIT,OP_NO) )
        newdata= cursor.fetchone() #op,bit,time(ms)
        z_score=float((newdata[2]-mean)/std)
        cursor.close()
        if (z_score > 1):
            result='NG'
        elif(z_score < -1):
            result='NG'
        else:
            result="OK"
        return OP_NO,STATS_BIT,result
    
    except:
        print('no data!!')
        alarm='nodata'
        return OP_NO,STATS_BIT,alarm

In [217]:
result=CalActuater(40,49)

no data!!


In [218]:
# init form of JSON is hardcodiing

In [219]:
import json
from collections import OrderedDict

In [222]:
LINE_group["LINE"] = "CETA31"
if result[0]==10:
    STATUS_BIT_group_OP10[result[1]]=result[2]
elif result[0]==20:
    STATUS_BIT_group_OP20[result[1]]=result[2]
elif result[0]==30:
    STATUS_BIT_group_OP30[result[1]]=result[2]
elif result[0]==40:
    STATUS_BIT_group_OP40[result[1]]=result[2]
elif result[0]==50:
    STATUS_BIT_group_OP50[result[1]]=result[2]    
else:
    print("error!!! check OP_NO") 

In [223]:
OP_group["10"]=STATUS_BIT_group_OP10
OP_group["20"]=STATUS_BIT_group_OP20
OP_group["30"]=STATUS_BIT_group_OP30
OP_group["40"]=STATUS_BIT_group_OP40
OP_group["50"]=STATUS_BIT_group_OP50
LINE_group["OP_group"]=OP_group
print(json.dumps(LINE_group, ensure_ascii=False, indent="\t") )

{
	"LINE": "CETA31",
	"OP_group": {
		"10": {},
		"20": {},
		"30": {
			"49": "nodata"
		},
		"40": {
			"49": "nodata"
		},
		"50": {}
	}
}


In [216]:

with open('CEPS31.json', 'w', encoding="utf-8") as make_file:
    json.dump(LINE_group, make_file, ensure_ascii=False, indent="\t")
